 # Create Binary Cold Wave Day S2S Reforecast Ensemble
 Version 19 January 2024, Selina Kiefer

### Input: csv-file
S2S reforecasts ensemble with absolute temperatures in csv-format
### Output: csv-file, png-files
S2S reforecast ensemble consisting of binary cold wave days in csv-format and plotted in png-format

#### Set the paths' to the defined functions, the style sheet for plotting and the configuration file and set its name

In [ ]:
# Set the path to the defined functions.
PATH_defined_functions = './Defined_Functions/'

In [ ]:
# Set the path and name of the style file which should be used for plotting.
style_file_for_plotting = './Style_File_Matplotlib.mplstyle'

In [ ]:
# Set the path and name of the configuration file.
PATH_configurations = './Configurations/'
ifile_configurations = 'Configurations_S2S_Reforecasts_Cold_Wave_Day_Ensemble.yaml'

#### Import the necessary python packages and functions

In [ ]:
# Import the necessary python packages.
import yaml
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Read in the necessary defined functions.
import sys
sys.path.insert(1, PATH_defined_functions)
from read_in_csv_data import *

#### Read in the style sheet for plotting

In [ ]:
# Load the style sheet to be used by matplotlib for plotting. This will update the plotting
# parameters to e.g. have the right font, font size and figure size. The latter is adjusted to
# the textwidth of the LaTeX-document in order to avoid re-scaling the plot and changing 
# thereby the font size again.
plt.style.use(style_file_for_plotting)
import warnings
warnings.simplefilter(action='ignore')

#### Read in the configuration file and the data specified in it

In [ ]:
# Read in the configuration file (nothing needs to be changed here).
with open(PATH_configurations+ifile_configurations) as f:
    config = yaml.safe_load(f)

In [ ]:
# Read in the S2S reforecasts ensemble with absolute temperature values.
df_input_data = read_in_csv_data(config['PATH_s2s_reforecast_ensemble'], config['ifile_s2s_reforecast_ensemble'])
df_input_data = df_input_data.drop(['index', 'Unnamed: 0'], axis=1)
df_input_data

In [ ]:
# Name the columns containing the ensemble members.
columns_reforecasts = ['0.0', '1.0', '2.0', '3.0', '4.0', '5.0', '6.0', '7.0', '8.0', '9.0', '10.0']

In [ ]:
# Read in the cold wave day thresholds.
df_cold_wave_thresholds = read_in_csv_data(config['PATH_cold_wave_thresholds'], config['ifile_cold_wave_thresholds'])
df_cold_wave_thresholds = df_cold_wave_thresholds.drop(['index', 'Unnamed: 0'], axis=1)
df_cold_wave_thresholds

#### Select the cold wave thresholds for the dates (month and day) present in the S2S reforecasts ensemble

In [ ]:
# Select from the S2S reforecast ensemble the same winter as used as the auxiliary date of the cold wave day
# thresholds.
df_for_thresholds = df_input_data.where(df_input_data['valid_time']>'2003-04-30')
df_for_thresholds = df_for_thresholds.where(df_for_thresholds['valid_time']<'2004-11-01')
df_for_thresholds = df_for_thresholds.dropna()

In [ ]:
# Find the dates which are present in both, the S2S reforecasts ensemble and the cold wave threshold data. If the 
# date is not present in both, add NaN to the list containing the dates so that the list has the same lenghts as
# the dataframe containing the cold wave thresholds.
joint_dates = []
l = 0

for i in range(len(df_cold_wave_thresholds['auxiliary_date'])):
    if df_cold_wave_thresholds['auxiliary_date'].iloc[i] == df_for_thresholds['valid_time'].iloc[l]:
        joint_dates.append(df_cold_wave_thresholds['auxiliary_date'].iloc[i])
        l = l+1
        if l>len(df_for_thresholds['valid_time'])-1:
            l = 0
    else:
        joint_dates.append(np.nan)

In [ ]:
# Append the list with the joint dates to the dataframe containing the cold wave thresholds. Drop all rows with
# NaN, leaving only the data for the joint dates in the dataframe.
df_cold_wave_thresholds['joint_dates'] = joint_dates
df_cold_wave_thresholds = df_cold_wave_thresholds.dropna()
df_cold_wave_thresholds = df_cold_wave_thresholds.drop(['joint_dates'], axis=1)

#### Calculate the cold wave days on the S2S reforecasts ensemble

In [ ]:
# Repeat the list with the cold wave threshold for every winter so that it has the same lengths as the dataframe
# containing the S2S reforecast ensemble with the absolute temperature values.
number_of_winters = len(df_input_data['valid_time'])/len(df_cold_wave_thresholds['threshold_cold_wave'])
cold_wave_thresholds = np.tile(df_cold_wave_thresholds['threshold_cold_wave'], int(number_of_winters))

In [ ]:
# Add the cold wave thresholds to the dataframe containing the S2S reforecast ensemble.
df_input_data['threshold_cold_wave'] = cold_wave_thresholds

In [ ]:
# Subtract the value of the cold wave threshold from the S2S reforecasts.
for k in columns_reforecasts:
    df_input_data[k] = df_input_data[k]-df_input_data['threshold_cold_wave']

In [ ]:
# Set all values fulfilling the cold wave day criterion to 1, the rest to 0.
for k in columns_reforecasts:
    df_input_data[k][df_input_data[k] >= 0] = 0
    df_input_data[k][df_input_data[k] < 0] = 1

#### Save the cold wave day S2S reforecast ensemble and visualize it

In [ ]:
# Save the ensemble as csv-file.
df_input_data.to_csv(config['PATH_output_files']+'S2S_Reforecast_Binary_Cold_Wave_Day_Ensemble_Lead_Time_'+config['lead_time']+'_2000_2020.csv')

In [ ]:
# For plotting, the fraction of ensemble member showing a cold wave day is calculated.
df_input_data['Fraction_Cold_Wave_Day'] = df_input_data[columns_reforecasts].values.mean(axis=1) 

In [ ]:
# Plot the cold wave day S2S reforecast ensemble for all winters for a plausibility check.
fig, ax = plt.subplots()
plt.plot(df_input_data['valid_time'], df_input_data['Fraction_Cold_Wave_Day'], marker='o', linestyle='', markersize=1)
ax.xaxis.set_major_locator(plt.MaxNLocator(15))
plt.xticks(rotation=45)
plt.xlabel('time')
plt.ylabel('Fraction of Ensemble Members')
plt.title('Cold Wave Days in ECMWF S2S Reforecasts, Lead Time '+config['lead_time'])
plt.savefig(config['PATH_plots']+'S2S_Reforecast_Binary_Cold_Wave_Day_Ensemble_Lead_Time_'+config['lead_time']+'_Winter_2000_2020.png', bbox_inches='tight')

In [ ]:
# Plot the cold wave day S2S reforecast ensemble for a single winter for a plausibility check.
df_s2s_reforecast_ensemble_winter_2011_2012 = df_input_data.iloc[572:623]
fig, ax = plt.subplots()
plt.plot(df_s2s_reforecast_ensemble_winter_2011_2012['valid_time'], df_s2s_reforecast_ensemble_winter_2011_2012['Fraction_Cold_Wave_Day'], marker='o', linestyle='', linewidth=1, markersize=3)
ax.xaxis.set_major_locator(plt.MaxNLocator(15))
plt.xticks(rotation=45)
plt.xlabel('time')
plt.ylabel('Fraction of Ensemble Members')
plt.title('Cold Wave Days in ECMWF S2S Reforecasts, Lead Time '+config['lead_time'])
plt.savefig(config['PATH_plots']+'S2S_Reforecast_Binary_Cold_Wave_Day_Ensemble_Lead_Time_'+config['lead_time']+'_Winter_2011_2012.png', bbox_inches='tight')

In [ ]:
# End of Program.